In [1]:
import numpy as np 
import pandas as pd 
import os
import csv
from mlxtend.frequent_patterns import apriori, association_rules 
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None)  

import warnings
warnings.filterwarnings('ignore')
#from apyori import apriori

In [2]:
#klasor_yolu = '1150haber/raw_texts/'
klasor_yolu = r'C:\Users\Barış\Desktop\Apriori\1150haber\raw_texts'
toplam_df = pd.DataFrame()

for klasor_adı, alt_klasorler, dosya_listesi in os.walk(klasor_yolu):
    for dosya_adı in dosya_listesi:
        if dosya_adı.endswith('.txt'):
            dosya_yolu = os.path.join(klasor_adı, dosya_adı)
        
        with open(dosya_yolu, 'r',encoding='unicode_escape') as dosya:
            icerik = dosya.read()
            kelimeler = icerik.split()
            df = pd.DataFrame({'Dosya': [dosya_adı]})
            for i, kelime in enumerate(kelimeler):
                df[f'Kelime{i+1}'] = kelime
                if i > 0:
                    df[f'Kelime{i}'] = kelimeler[i-1]
            toplam_df = pd.concat([toplam_df, df], ignore_index=True)         

In [3]:
toplam_df.head(10)

Dosya      Kelime1   Kelime2        Kelime3         Kelime4  \
0    1.txt       ANKARA         -           Türk      Telekom'un   
1   10.txt       TAHRAN         -           Ýran   Cumhurbaþkaný   
2  100.txt  Türkiye'den       zam      kararýyla          ilgili   
3  101.txt       Ankara    Sanayi          Odasý           (ASO)   
4  102.txt      Yatýrým     yapan      kazanacak         Anadolu   
5  103.txt       Ziraat  Bankasý,         Hazine      tarafýndan   
6  104.txt        Japan   Tobacco  International          (JTI),   
7  105.txt     Piyasada    petrol     gerginliði           Devam   
8  106.txt       SSKya    pahalý           ilaç         satarak   
9  107.txt        Ýzmir     Adnan       Menderes  Havalimanýnda   

        Kelime5     Kelime6     Kelime7       Kelime8    Kelime9  \
0       indirim  reklamlarý      Sanayi            ve    Ticaret   
1      Muhammed     Hatemi,     yabancý  þirketlerden      gelen   
2       yapýlan  açýklamada      zammýn      ardýnda,  hükümetin   
3       Baþkaný       Zafer   Çaðlayan,   Türkiye'nin        bir   
4         Hayat   Emeklilik       Genel        Müdürü    Mustafa   
5  belirlenecek     tarihte  baþlayacak          olan     üçüncü   
6         geçen       hafta        özel       tüketim    vergisi   
7          eden        cari        açýk     endiþesi,     petrol   
8       devleti      zarara   uðrattýðý  gerekçesiyle   hakkýnda   
9           bir  güvenlik     eziyeti    yaþanýyor.       THY,   

        Kelime10     Kelime11      Kelime12       Kelime13      Kelime14  \
0   Bakanlýðý'na     takýldý.        Sanayi             ve       Ticaret   
1            tüm   itirazlara       raðmen,        ülkenin        ikinci   
2     geçtiðimiz        hafta       baþýnda   sigaralardan        alýnan   
3       numaralý       sorunu          olan       istihdam          için   
4            Su,     Bireysel     Emeklilik    Sistemi'nde         (BES)   
5         taksit         nema  ödemeleriyle         ilgili         kurum   
6          (ÖTV)  oranlarýnda       yapýlan   deðiþikliðin     ardýndan,   
7  fiyatlarýndak       yüksek         seyir             ve  uluslararasý   
8     soruþturma   baþlatýlan      Rocheun  Ýstanbuldaki         genel   
9        transit    yolcularý            ve             iç           hat   

           Kelime15       Kelime16       Kelime17         Kelime18  \
0         Bakanlýðý    Tüketicinin             ve        Rekabetin   
1              özel         sektör            cep         telefonu   
2        vergilerle         ilgili        sistemi          tümüyle   
3  'hýzlandýrýlmýþ'       önlemler       alýnmasý      gerektiðini   
4           katýlým     paylarýnýn          hangi           oranda   
5                ve         iþyeri          toplu  baþvurularýnýn,   
6              orta          fiyat  kategorisinde              yer   
7      borsalardaki  olumsuzluklar     piyasalarý         rahatsýz   
8       müdürlüðüne         baskýn     düzenleyen           polis,   
9       yolcularýný           ayrý     otobüslere     bindirirken,   

          Kelime19            Kelime20       Kelime21     Kelime22  \
0        Korunmasý               Genel      Müdürlüðü   bünyesinde   
1        lisansýný          Turkcell'e  vereceklerini           ve   
2  deðiþtirmesiyle              ilgili     geliþmeler    bulunduðu   
3         söyledi.                 DSP          Genel      Baþkaný   
4         yatýrýma  yönlendirildiðinin         önemli     olduðunu   
5               18         Aðustostan       itibaren        kabul   
6             alan               ürünü        Winston  sigarasýnýn   
7            etti.              Borsa,           günü       sadece   
8             ilaç                alým      ihalesine     katýlmak   
9         yolcular                   1           saat        süren   

      Kelime23        Kelime24    Kelime25      Kelime26        Kelime27  \
0          yer            alan      Reklam       Kur

In [4]:
print(toplam_df.shape)

(1150, 1488)


In [ ]:
istenmeyen_kelimeler = ["ve", "bir", "nan", "de", "da", "bu", "ile", "için", "olarak", "Bu", "çok", "olan", "ise", "-", "her", 
                        "gibi", "kadar", "bile"]

records = []
for i in range(0, toplam_df.shape[0]):
    # top_df.values ile değerlere ulaşılır, her satırı kontrol ederek istenmeyen kelimeleri filtrele
    row = []
    for j in range(1, toplam_df.shape[1]):
        value = str(toplam_df.values[i, j])
        # Noktalama işaretlerini kaldır
        cleaned_value = value.rstrip(".,;:!?'\"").replace("ð", "ğ").replace("þ", "ş").replace("ý", "ı")
        if value not in istenmeyen_kelimeler:
            row.append(cleaned_value)
    records.append(row)


In [ ]:
print(records)

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
te=TransactionEncoder()
te_data=te.fit(records).transform(records)
df=pd.DataFrame(te_data,columns=te.columns_)
df.head()

In [ ]:
freq_items=apriori(df,min_support=0.06,use_colnames=True,verbose=1)

In [ ]:
freq_items_sorted = freq_items.sort_values(by='support', ascending=False)
freq_items_sorted.head(50)

In [ ]:
rules=association_rules(freq_items,metric="confidence",min_threshold=0.6)
rules=rules.sort_values(["confidence","lift"],ascending=[False,False])
rules